In [ ]:
# pip install requests python-dotenv

In [1]:
import requests
import pandas as pd
import csv
import webbrowser
from datetime import datetime, timedelta
from flask import Flask, request, redirect
from dotenv import load_dotenv
import os

In [2]:
# Obtenerlos desde un archivo .env
# Cargar el archivo .env
load_dotenv('api_meli_colch.env')

# Obtener los valores de las variables de entorno
app_id = os.getenv("APP_ID")
client_secret = os.getenv("CLIENT_SECRET")
redirect_uri = os.getenv("REDIRECT_URI")


In [4]:
# Abrir automaticamente la página web y copiar y pegar el Auth_code MANUALMENTE

auth_url = f'https://auth.mercadolibre.com.mx/authorization?response_type=code&client_id={app_id}&redirect_uri={redirect_uri}'
print("Visita esta URL para autorizar la aplicación:", auth_url)
webbrowser.open(auth_url)

Visita esta URL para autorizar la aplicación: https://auth.mercadolibre.com.mx/authorization?response_type=code&client_id=8261166610203184&redirect_uri=https://adseller.mx


True

In [5]:
auth_code = 'TG-66450396d250c3000176ddbc-1336637999'

In [6]:
def get_access_token(auth_code):
    # URL para intercambiar el código por un token de acceso
    token_url = 'https://api.mercadolibre.com/oauth/token'

    # Detalles para la petición de intercambio de token
    token_data = {
        'grant_type': 'authorization_code',
        'client_id': app_id,
        'client_secret': client_secret,
        'code': auth_code,
        'redirect_uri': redirect_uri
    }

    # Realizar la petición POST para obtener el token
    response = requests.post(token_url, data=token_data)
    if response.status_code == 200:
        access_token = response.json().get('access_token')
        print("Token de acceso obtenido:", access_token)
        return access_token
    else:
        print("Error al obtener el token de acceso:", response.status_code, response.text)
        return None

# Llamando a la función y almacenando el resultado
access_token = get_access_token(auth_code)


Token de acceso obtenido: APP_USR-8261166610203184-051514-72a18204ab6b9d9bf5ca45456cf73547-1336637999


In [13]:
def get_user_id(access_token):
    url = 'https://api.mercadolibre.com/users/me'
    headers = {'Authorization': f'Bearer {access_token}'}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        user_id = response.json()['id']
        print("ID de usuario obtenido:", user_id)
        return user_id
    else:
        print("Error al obtener el ID de usuario:", response.status_code, response.text)
        return None

user_id = get_user_id(access_token)


ID de usuario obtenido: 1336637999


In [17]:
def get_all_orders(user_id, access_token):
    # Calcular las fechas para los últimos 6 meses
    end_date = datetime.now()
    start_date = end_date - timedelta(days=180)  # Aproximadamente 6 meses

    # Formatear las fechas en el formato adecuado (YYYY-MM-DD)
    date_from = start_date.strftime('%Y-%m-%d')
    date_to = end_date.strftime('%Y-%m-%d')

    offset = 0
    limit = 50  # Ajustar según las limitaciones de la API
    all_orders = []

    while True:
        url = (f'https://api.mercadolibre.com/orders/search?seller={user_id}'
               f'&order.date_created.from={date_from}T00:00:00.000-00:00'
               f'&order.date_created.to={date_to}T23:59:59.999-00:00'
               f'&offset={offset}&limit={limit}')
        headers = {'Authorization': f'Bearer {access_token}'}
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            orders = response.json().get('results', [])
            all_orders.extend(orders)

            # Verificar si hay más órdenes para obtener
            if len(orders) < limit:
                break
            else:
                offset += limit
        else:
            print("Error al obtener las compras:", response.status_code, response.text)
            break

    print(f"Se han obtenido {len(all_orders)} órdenes desde {date_from} hasta {date_to}.")
    return all_orders

orders = get_all_orders(user_id, access_token)

Se han obtenido 638 órdenes desde 2023-11-17 hasta 2024-05-15.


In [18]:
def save_orders_to_csv(orders):
    if not orders:
        print("No hay datos de órdenes para guardar.")
        return

    # Definir las claves que queremos guardar en el CSV
    keys = [
        'id', 'status', 'date_created', 'date_closed', 'total_amount', 'currency_id',
        'buyer_id', 'buyer_nickname', 'buyer_first_name', 'buyer_last_name',
        'item_id', 'item_title', 'quantity', 'unit_price',
        'payment_id', 'transaction_amount', 'payment_status'
    ]

    with open('data/orders.csv', 'w', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, fieldnames=keys)
        dict_writer.writeheader()

        # Asegurar que todos los diccionarios tengan todas las claves
        for order in orders:
            for item in order.get('order_items', []):
                for payment in order.get('payments', []):
                    row = {
                        'id': order.get('id'),
                        'status': order.get('status'),
                        'date_created': order.get('date_created'),
                        'date_closed': order.get('date_closed'),
                        'total_amount': order.get('total_amount'),
                        'currency_id': order.get('currency_id'),
                        'buyer_id': order['buyer'].get('id'),
                        'buyer_nickname': order['buyer'].get('nickname'),
                        'buyer_first_name': order['buyer'].get('first_name'),
                        'buyer_last_name': order['buyer'].get('last_name'),
                        'item_id': item['item'].get('id'),
                        'item_title': item['item'].get('title'),
                        'quantity': item.get('quantity'),
                        'unit_price': item.get('unit_price'),
                        'payment_id': payment.get('id'),
                        'transaction_amount': payment.get('transaction_amount'),
                        'payment_status': payment.get('status')
                    }
                    dict_writer.writerow(row)

    print("Datos de órdenes guardados en 'orders.csv'.")


orders = get_all_orders(user_id, access_token)
if orders:
    save_orders_to_csv(orders)

Se han obtenido 638 órdenes desde 2023-11-17 hasta 2024-05-15.
Datos de órdenes guardados en 'orders.csv'.


Mandar a AWS - S3